In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

llm.invoke("HI")